# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'
#y0 = '2013'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

#north_pumas = ['0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP'

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
1000000 = 1.000000

FHINCP - income flag - 1: yes | https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

GRNTP - gross rent (monthly) (not 0: n/a,not paying rent, owned)
https://api.census.gov/data/2021/acs/acs1/pums/variables/GRNTP.json

In [4]:
#AMI bands in 2021 by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [5]:
#Housing costs affordable to different AMI bands based on household size
unit_afford = {'1':[0,415,691,1106,1383,1000000000],\
              '2':[0,475,790,1265,1580,1000000000],\
              '3':[0,549,888,1423,1775,1000000000],\
              '4':[0,663,988,1580,1975,1000000000],\
              '5':[0,776,1068,1708,2135,1000000000],\
              '6':[0,890,1146,1834,2293,1000000000],\
              '7':[0,1003,1225,1960,2450,1000000000],\
              '8':[0,1117,1304,2450,2608,1000000000]}

inc_lbls = ['u30_ami','30_50_ami','50_80_ami','80_100_ami','o100_ami']

In [6]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [7]:
df = get.get_puma(sample,y1,data_cols)

In [8]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [9]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)
df['HINCP'] = df.ADJINC * df.HINCP

In [10]:
dff = df[(df.GRNTP!=0)].copy()

In [11]:
dff['hou_cost'] = dff.GRNTP
dff['aff_cost'] = np.where(dff.HINCP>=1,(dff.HINCP*0.333)/12,0)

In [12]:
#who is the unit affordable to based on the rent
dff['unit_aff'] = np.where(dff.HHSz=='1',pd.cut(dff['hou_cost'],bins=unit_afford['1'],labels=inc_lbls),\
                  np.where(dff.HHSz=='2',pd.cut(dff['hou_cost'],bins=unit_afford['2'],labels=inc_lbls),\
                  np.where(dff.HHSz=='3',pd.cut(dff['hou_cost'],bins=unit_afford['3'],labels=inc_lbls),\
                  np.where(dff.HHSz=='4',pd.cut(dff['hou_cost'],bins=unit_afford['4'],labels=inc_lbls),\
                  np.where(dff.HHSz=='5',pd.cut(dff['hou_cost'],bins=unit_afford['5'],labels=inc_lbls),\
                  np.where(dff.HHSz=='6',pd.cut(dff['hou_cost'],bins=unit_afford['6'],labels=inc_lbls),\
                  np.where(dff.HHSz=='7',pd.cut(dff['hou_cost'],bins=unit_afford['7'],labels=inc_lbls),\
                  np.where(dff.HHSz=='8',pd.cut(dff['hou_cost'],bins=unit_afford['8'],labels=inc_lbls),''))))))))

In [13]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,WGTP1,WGTP2,WGTP3,...,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,HHSz,hou_cost,aff_cost,unit_aff
3612,0400120,177353.6016,3.0,42.0,1.029928,1000000.0,2460.0,74.0,13.0,71.0,...,71.0,78.0,42.0,44.0,43.0,42.0,3,2460.0,4921.562444,o100_ami
3637,0400119,55307.1336,7.0,172.0,1.029928,1000000.0,1405.0,70.0,81.0,157.0,...,54.0,270.0,152.0,164.0,282.0,65.0,7,1405.0,1534.772957,50_80_ami
3639,0400116,70344.0824,2.0,177.0,1.029928,1000000.0,1808.0,244.0,245.0,293.0,...,174.0,242.0,173.0,299.0,161.0,194.0,2,1808.0,1952.048287,o100_ami


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [14]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [15]:
drop_cols = ['HINCP','NP','ADJINC','ADJHSG','GRNTP','HHSz','hou_cost']

In [16]:
table = dff.copy().drop(columns=drop_cols)

In [17]:
table = table.groupby(['GEO_ID','unit_aff']).sum().reset_index()

In [18]:
table.WGTP.sum()

246922.0

In [19]:
table.head()

,GEO_ID,unit_aff,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,WGTP7,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,aff_cost
0,0400112,50_80_ami,1136.0,1412.0,1602.0,1116.0,1110.0,1225.0,1407.0,1510.0,...,805.0,1311.0,1281.0,943.0,1299.0,939.0,1036.0,1055.0,1242.0,13138.456769
1,0400112,80_100_ami,1112.0,1559.0,1087.0,489.0,625.0,827.0,1288.0,798.0,...,1316.0,1170.0,1336.0,864.0,1127.0,1215.0,1492.0,1188.0,680.0,10429.025180
2,0400112,o100_ami,9471.0,9025.0,9167.0,9805.0,9880.0,9569.0,9097.0,9842.0,...,9233.0,10054.0,9029.0,8506.0,10126.0,7599.0,8653.0,9380.0,9739.0,156515.974713
3,0400112,u30_ami,118.0,113.0,34.0,122.0,117.0,35.0,201.0,117.0,...,125.0,36.0,200.0,118.0,116.0,116.0,200.0,115.0,35.0,1663.385216
4,0400113,30_50_ami,570.0,478.0,414.0,499.0,584.0,674.0,376.0,274.0,...,665.0,693.0,576.0,693.0,250.0,552.0,639.0,773.0,684.0,7359.479265


In [20]:
table_2 = table.copy().groupby(['GEO_ID','unit_aff']).sum().reset_index()
table_2 = make_est(table_2)
table_2 = table_2.drop(columns=repwts)

In [21]:
table_2 = pd.pivot_table(table_2,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['unit_aff'],aggfunc=np.sum).reset_index()

In [22]:
table_2

GEO_ID        hh                                            hh_CV  \
unit_aff          30_50_ami 50_80_ami 80_100_ami o100_ami u30_ami  30_50_ami   
0         0400112       NaN    1136.0     1112.0   9471.0   118.0        NaN   
1         0400113     570.0    2984.0     3436.0   5155.0   282.0  23.151115   
2         0400114    1582.0    7873.0     4224.0   4718.0   516.0  13.104895   
3         0400115    1387.0    7674.0     6030.0   4051.0   170.0  12.200952   
4         0400116    2957.0   13171.0     5569.0   4919.0  1280.0  10.185886   
5         0400117    1509.0    7916.0     6400.0   7460.0   602.0  15.187794   
6         0400118    3796.0   10980.0     7682.0   8310.0  2679.0   8.307527   
7         0400119    1697.0    4314.0     2593.0   1267.0  1483.0  11.266442   
8         0400120     141.0    1316.0     4377.0   6329.0   342.0  43.029785   
9         0400121    1839.0    5073.0     1645.0   1598.0   640.0  11.094945   
10        0400122    1004.0    9971.0     1681.0    471.0   982.0  16.137529   
11        0400123    1350.0    8403.0     3160.0   1044.0   535.0  14.637354   
12        0400125    1272.0    8331.0     2891.0   1712.0   940.0  14.880621   
13        0400128     547.0    4590.0     4181.0   2680.0   181.0  26.559527   
14        0400129       NaN    2380.0     2698.0   3496.0    49.0        NaN   

                                                          hh_MOE               \
unit_aff  50_80_ami 80_100_ami   o100_ami    u30_ami   30_50_ami    50_80_ami   
0         21.908319  22.510170   5.759228  42.787027         NaN   673.471468   
1          9.137122   9.181666   6.995259  40.708814  357.090722   737.802397   
2          5.956305   7.836763   8.520792  16.509424  561.011568  1268.963039   
3          6.585900   7.789635   7.900819  33.585426  457.933034  1367.630261   
4          3.702247   7.080197   5.752966  15.579455  815.045667  1319.519936   
5          6.306884   6.360582   5.789441  17.115550  620.177108  1350.990890   
6          4.779948   5.745264   5.334996   9.204329  853.355027  1420.225614   
7          7.094125  10.498988  13.106651  14.762001  517.369041   828.153395   
8         15.853775   8.075740   6.451100  31.874984  164.179942   564.573370   
9          6.781811  15.072433  14.249874  22.154047  552.126612   930.984331   
10         4.216442  16.822779  22.631187  16.454305  438.432312  1137.670987   
11         5.980251  10.382489  16.170188  26.668093  534.722109  1359.833034   
12         5.916850  10.605233  13.135429  21.422948  512.200475  1333.888319   
13        11.998857   9.538653  10.846965  29.704582  393.132974  1490.336614   
14        11.225075   9.538941   8.417448  44.836793         NaN   722.932958   

                                                
unit_aff   80_100_ami     o100_ami     u30_ami  
0          677.353494  1476.018887  136.623662  
1          853.702304   975.807705  310.648570  
2          895.761561  1087.851790  230.522555  
3         1271.060428   866.096210  154.501103  
4         1066.975298   765.773650  539.627451  
5         1101.561141  1168.711259  278.816946  
6         1194.307301  1199.684160  667.262370  
7          736.684879   449.365933  592.404276  
8          956.512618  1104.843220  294.990402  
9          670.935942   616.197052  383.676194  
10         765.239298   288.443022  437.242965  
11         887.810668   456.822379  386.080221  
12         829.659953   608.526946  544.927697  
13        1079.192813   786.637856  145.490230  
14         696.424257   796.312795   59.451447

In [23]:
table_2.to_excel('output/pums_rent_revised_june.xlsx')